In [1]:
import os, sys
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

sys.path.append('/Users/adaly/Documents/Python/gridnext/src/')
from image_datasets import PatchDataset, PatchGridDataset
from training import train_spotwise, train_gridwise
from densenet import DenseNet

In [2]:
# Paths to count, image data
data_dir = '/Users/adaly/Desktop/BA44/'

meta = pd.read_csv(os.path.join(data_dir, 'Splotch_Metadata.tsv'), sep='\t', header=0)

image_files = np.array([os.path.join(data_dir, ifile.replace('HE', 'patchdata')).replace('.jpg', '/')
                        for ifile in meta['Image file']])
annot_files = np.array([os.path.join(data_dir, 
                                     afile.replace('annotations', 'annotations_splotch')).replace('csv', 'tsv') 
                        for afile in meta['Annotation file']])

annot_names = pd.read_csv(annot_files[0], index_col=0, header=0, sep='\t').index

individuals = np.unique(meta['Level 3'])
partitions = [individuals[:3], individuals[3:6], individuals[6:9], individuals[9:]]

In [3]:
print(image_files[:2])
print(annot_files[:2])
test = PatchDataset(image_files[:2], annot_files[:2])

['/Users/adaly/Desktop/BA44/patchdata/V003_V19T19-041-A1/'
 '/Users/adaly/Desktop/BA44/patchdata/V003_V19T19-041-B1/']
['/Users/adaly/Desktop/BA44/annotations_splotch/V003-CGND-HRA-02744-A.tsv'
 '/Users/adaly/Desktop/BA44/annotations_splotch/V003-CGND-HRA-02744-B.tsv']


In [ ]:
# DenseNet121 preprocessing transform
ppx = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_hist_part, val_hist_par = [], []

for i, p in enumerate(partitions):
    print('Test Partition: %s' % ', '.join(p))
    val_inds = np.array([indiv in p for indiv in meta['Level 3']])
    train_inds = np.logical_not(val_inds)
    
    val_set = PatchDataset(image_files[val_inds], annot_files[val_inds], img_transforms=ppx)
    train_set = PatchDataset(image_files[train_inds], annot_files[train_inds], img_transforms=ppx)
    
    dataloader_spots = {
        'train': DataLoader(train_set, batch_size=16, shuffle=True),
        'val': DataLoader(val_set, batch_size=16)
    }
    
    patch_clf = DenseNet(num_classes=len(annot_names), small_inputs=False, efficient=False,
        growth_rate=32, block_config=(6, 12, 24, 16), num_init_features=64, bn_size=4, drop_rate=0)
    
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(patch_clf.parameters(), lr=1e-4)

    patch_clf, val_hist, train_hist = train_spotwise(patch_clf, dataloader_spots, loss, optimizer, 
                                                     100, display=False,
                                                     outfile='../models/densenet_ba44_p%d.pth' % i)
    train_hist_part.append(train_hist)
    val_hist_part.append(val_hist)

    fig = plt.figure()
    plt.plot(train_hist, label='train')
    plt.plot(val_hist, label='val')
    plt.xlabel('Training epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

Test Partition: BA44_SD004/16, BA44_SD008/13, BA44_SD011/18
/Users/adaly/Desktop/BA44/annotations_splotch/V028-CGND-HRA-02749-A.tsv 97_5 improper annotation
/Users/adaly/Desktop/BA44/annotations_splotch/V028-CGND-HRA-02749-A.tsv 47_55 improper annotation
Epoch 0/99
----------
